<a href="https://colab.research.google.com/github/yong2khoo-lm/Django-tutorial/blob/main/Lab_Detect_FFB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 577 kB 6.3 MB/s 
     |████████████████████████████████| 840 kB 56.7 MB/s 
     |████████████████████████████████| 60.9 MB 1.6 MB/s 
     |████████████████████████████████| 10.9 MB 38.5 MB/s 
     |████████████████████████████████| 238 kB 55.0 MB/s 
     |████████████████████████████████| 77 kB 4.8 MB/s 
     |████████████████████████████████| 1.3 MB 62.2 MB/s 
     |████████████████████████████████| 1.1 MB 12.6 MB/s 
     |████████████████████████████████| 3.4 MB 52.7 MB/s 
     |████████████████████████████████| 128 kB 72.6 MB/s 
     |████████████████████████████████| 87 kB 6.9 MB/s 
     |████████████████████████████████| 25.3 MB 1.4 MB/s 
     |████████████████████████████████| 497.9 MB 4.2 kB/s 
     |████████████████████████████████| 352 kB 58.4 MB/s 
     |████████████████████████████████| 1.4 MB 52.0 MB/s 
     |████████████████████████████████| 462 kB 67.8 MB/s 
     |████████████████████████████████| 5.8 MB 46.6 MB/s 
     |███████████

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

# New Section

In [ ]:
import os.path
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    '/content/drive/MyDrive/Lab/FFB_202210/train',
    '/content/drive/MyDrive/Lab/FFB_202210/train',
    ['ffb']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    '/content/drive/MyDrive/Lab/FFB_202210/validate',
    '/content/drive/MyDrive/Lab/FFB_202210/validate',
    ['ffb']
)



In [ ]:
type(train_data)

tensorflow_examples.lite.model_maker.core.data_util.object_detector_dataloader.DataLoader

In [ ]:
VERSION_EFF_LITE = 3
spec = model_spec.get(f'efficientdet_lite{VERSION_EFF_LITE}')

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=25, validation_data=val_data)

Epoch 1/25
87/87 [==============================] - 94s 467ms/step - det_loss: 1.4053 - cls_loss: 0.8638 - box_loss: 0.0108 - reg_l2_loss: 0.0926 - loss: 1.4979 - learning_rate: 0.0065 - gradient_norm: 3.6705 - val_det_loss: 0.7619 - val_cls_loss: 0.4538 - val_box_loss: 0.0062 - val_reg_l2_loss: 0.0927 - val_loss: 0.8546
Epoch 2/25
87/87 [==============================] - 39s 445ms/step - det_loss: 1.1059 - cls_loss: 0.6314 - box_loss: 0.0095 - reg_l2_loss: 0.0927 - loss: 1.1986 - learning_rate: 0.0050 - gradient_norm: 3.6883 - val_det_loss: 0.6089 - val_cls_loss: 0.4210 - val_box_loss: 0.0038 - val_reg_l2_loss: 0.0927 - val_loss: 0.7016
Epoch 3/25
87/87 [==============================] - 40s 465ms/step - det_loss: 1.0527 - cls_loss: 0.6043 - box_loss: 0.0090 - reg_l2_loss: 0.0927 - loss: 1.1454 - learning_rate: 0.0049 - gradient_norm: 3.2553 - val_det_loss: 0.4848 - val_cls_loss: 0.3545 - val_box_loss: 0.0026 - val_reg_l2_loss: 0.0927 - val_loss: 0.5775
Epoch 4/25
87/87 [=============

In [ ]:
model.evaluate(val_data)

1/1 [==============================] - 7s 7s/step



{'AP': 0.7595155,
 'AP50': 1.0,
 'AP75': 0.95383936,
 'APs': -1.0,
 'APm': 0.81815183,
 'APl': 0.75698835,
 'ARmax1': 0.3074074,
 'ARmax10': 0.8074074,
 'ARmax100': 0.8074074,
 'ARs': -1.0,
 'ARm': 0.8428571,
 'ARl': 0.80212766,
 'AP_/ffb': 0.7595155}

In [ ]:
{'AP': 0.75924647,
 'AP50': 0.99964,
 'AP75': 0.97592854,
 'APs': -1.0,
 'APm': 0.71507347,
 'APl': 0.7652959,
 'ARmax1': 0.30555555,
 'ARmax10': 0.8111111,
 'ARmax100': 0.8111111,
 'ARs': -1.0,
 'ARm': 0.7714286,
 'ARl': 0.81702125,
 'AP_/ffb': 0.75924647}

In [ ]:
tflite_fn = f'android-ffb-count{VERSION_EFF_LITE}.tflite'
model.export(export_dir='.', tflite_filename=tflite_fn)

In [ ]:
model.evaluate_tflite(tflite_fn, val_data) 

20/20 [==============================] - 359s 18s/step



{'AP': 0.7364692,
 'AP50': 1.0,
 'AP75': 0.92959297,
 'APs': -1.0,
 'APm': 0.73640263,
 'APl': 0.7369886,
 'ARmax1': 0.2888889,
 'ARmax10': 0.7925926,
 'ARmax100': 0.7925926,
 'ARs': -1.0,
 'ARm': 0.7714286,
 'ARl': 0.79574466,
 'AP_/ffb': 0.7364692}

In [ ]:
from google.colab import files
files.download(tflite_fn)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:


from PIL import Image

INPUT_IMAGE_URL = "http://download.tensorflow.org/example_images/android_figurine.jpg" #@param {type:"string"}
DETECTION_THRESHOLD = 0.5 #@param {type:"number"}
TFLITE_MODEL_PATH = "android-ffb-count.tflite" #@param {type:"string"}

TEMP_FILE = '/tmp/image.png'

!wget -q -O $TEMP_FILE $INPUT_IMAGE_URL
image = Image.open(TEMP_FILE).convert('RGB')
image.thumbnail((512, 512), Image.ANTIALIAS)
image_np = np.asarray(image)

# Load the TFLite model
options = ObjectDetectorOptions(
      num_threads=4,
      score_threshold=DETECTION_THRESHOLD,
)
detector = ObjectDetector(model_path=TFLITE_MODEL_PATH, options=options)

# Run object detection estimation using the model.
detections = detector.detect(image_np)

# Draw keypoints and edges on input image
image_np = visualize(image_np, detections)

# Show the detection result
Image.fromarray(image_np)

NameError: ignored